In [ ]:
%pip install tensorflow matplotlib

In [ ]:
%pip install matplotlib

In [1]:
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
with open('./custom_mosi.pickle', 'rb') as fp:
    data = pickle.load(fp)

In [8]:
# Model Definition

# Modality Specific Module
class ModalitySpecificModule:
    @staticmethod
    def extract_modality_specific_interactions(self, input_layer):
        bi_gru_output = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(input_layer)
        msl_l1 = tf.keras.layers.Dense(128, activation='relu')(bi_gru_output)
        msl_output = tf.keras.layers.Dense(64, activation='relu')(msl_l1)
        return msl_output

    # Define inputs
    text_input = tf.keras.Input(shape=(300,))
    audio_input = tf.keras.Input(shape=(1585,))
    visual_input = tf.keras.Input(shape=(35,))

    @staticmethod
    def compute(text_input, audio_input, visual_input):
        # Extract Modality Specific Interactions
        text_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(text_input)
        audio_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(audio_input)
        visual_msm_output = ModalitySpecificModule.extract_modality_specific_interactions(visual_input)
        return text_msm_output, audio_msm_output, visual_msm_output


# Dense Multimodal Fusion Module
class DenseMultimodalFusionModule:
    residual_features = []

    @staticmethod
    def df_block(df_input1, df_input2, df_input3):
        df_output1 = tf.keras.layers.Dense(64, activation='relu')(df_input1)
        df_output2 = tf.keras.layers.Dense(64, activation='relu')(df_input2)
        df_output3 = tf.keras.layers.Dense(64, activation='relu')(df_input3)
        return df_output1, df_output2, df_output3
        
    @staticmethod
    def dense_fusion_layer(f1, f2, f3):
        df_input1 = tf.keras.layers.Concatenate()([f1, f3])
        df_input2 = tf.keras.layers.Concatenate()([f1, f2])
        df_input3 = tf.keras.layers.Concatenate()([f2, f3])
        r = tf.add(f1, f2, f3)
        DenseMultimodalFusionModule.residual_features.append(r)
        fusion_output = DenseMultimodalFusionModule.df_block(df_input1, df_input2, df_input3)
        return fusion_output
    
    @staticmethod
    def compute(f1_0, f2_0, f3_0):
        f1_1, f2_1, f3_1 = DenseMultimodalFusionModule.dense_fusion_layer(f1_0, f2_0, f3_0)
        f1_2, f2_2, f3_2 = DenseMultimodalFusionModule.dense_fusion_layer(f1_1, f2_1, f3_1)
        f1_3, f2_3, f3_3 = DenseMultimodalFusionModule.dense_fusion_layer(f1_2, f2_2, f3_2)
        r = tf.add(f1_3, f2_3, f3_3)
        DenseMultimodalFusionModule.residual_features.append(r)
        return DenseMultimodalFusionModule.residual_features


# Multimodal Residual Module
class MultimodalResidualModule:
    @staticmethod
    def compute(residual_features):
        final_residual_feature = residual_features[0]
        for r in residual_features[1:]:
            final_residual_feature = tf.add(final_residual_feature, r)
        return final_residual_feature

# Sentiment Classification Module
class SentimentClassificationModule:
    @staticmethod
    def compute(residual_feature):
        output_l1 = tf.keras.layers.Dense(64, activation='relu')(residual_feature)
        output_l2 = tf.keras.layers.Dense(32, activation='relu')(output_l1)
        output_l3 = tf.keras.layers.Dense(16, activation='relu')(output_l2)
        sentiment_output = tf.keras.layers.Dense(1, activation='relu')(output_l3)
        return sentiment_output

text_input, audio_input, visual_input = ModalitySpecificModule.text_input, ModalitySpecificModule.audio_input, ModalitySpecificModule.visual_input
text_msm_output, audio_msm_output, visual_msm_output = ModalitySpecificModule.compute(text_input, audio_input, visual_input)
residual_features = DenseMultimodalFusionModule.compute(text_msm_output, audio_msm_output, visual_msm_output)
final_residual_feature = MultimodalResidualModule.compute(residual_features)
output = SentimentClassificationModule.compute(final_residual_feature)


model = tf.keras.Model(inputs=[text_input, audio_input, visual_input], outputs=output)

(35,)
